# MCIS6273 Data Mining (Prof. Maull) / Spring 2025 / HW0

## Chocolate Review Data Engineering

### 1. Data Loading and Inspection 

In [1]:
# Import Required Libraries
import pandas as pd
import os


In [2]:
# Ensure the 'data' folder exists
os.makedirs('data', exist_ok=True)

In [3]:
# Load Data
print("Loading dataset...")
df = pd.read_csv("2024_flavors_of_cacoa.csv")
print("Data Loaded Successfully!\n")
df.head()

Loading dataset...
Data Loaded Successfully!



,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,2542,5150,U.S.A.,2021,India,"Anamalai, batch 1",68%,"3- B,S,C","milk brownie, macadamia,chewy",3.50
4,2546,5150,U.S.A.,2021,Uganda,"Semuliki Forest, batch 1",80%,"3- B,S,C","mildly bitter, basic cocoa, fatty",3.25


In [4]:
print("Data information\n")
df.info()

Data information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2789 entries, 0 to 2788
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   REF                               2789 non-null   int64  
 1   Company (Manufacturer)            2789 non-null   object 
 2   Company Location                  2789 non-null   object 
 3   Review Date                       2789 non-null   int64  
 4   Country of Bean Origin            2789 non-null   object 
 5   Specific Bean Origin or Bar Name  2789 non-null   object 
 6   Cocoa Percent                     2789 non-null   object 
 7   Ingredients                       2789 non-null   object 
 8   Most Memorable Characteristics    2789 non-null   object 
 9   Rating                            2789 non-null   float64
dtypes: float64(1), int64(2), object(7)
memory usage: 218.0+ KB


In [5]:
# Display column names to verify
print("Checking column names...")
print(df.columns)


Checking column names...
Index(['REF', 'Company (Manufacturer)', 'Company Location', 'Review Date',
       'Country of Bean Origin', 'Specific Bean Origin or Bar Name',
       'Cocoa Percent', 'Ingredients', 'Most Memorable Characteristics',
       'Rating'],
      dtype='object')


### 2.Data Transformation

In [6]:
# Ensure Correct Column Naming
df.columns = df.columns.str.strip()  # Remove extra spaces

# Rename 'Cocoa Percent' to 'Cacao Percent' for consistency
if "Cocoa Percent" in df.columns:
    df.rename(columns={"Cocoa Percent": "Cacao Percent"}, inplace=True)
    print("Renamed 'Cocoa Percent' to 'Cacao Percent'")

# Display updated column names
print("Updated column names:", df.columns)

Renamed 'Cocoa Percent' to 'Cacao Percent'
Updated column names: Index(['REF', 'Company (Manufacturer)', 'Company Location', 'Review Date',
       'Country of Bean Origin', 'Specific Bean Origin or Bar Name',
       'Cacao Percent', 'Ingredients', 'Most Memorable Characteristics',
       'Rating'],
      dtype='object')


In [7]:
## 1. Transform 'Cacao Percent' to Float
print("Transforming 'Cacao Percent' column...")
df["Cacao Percent"] = df["Cacao Percent"].astype(str)  # Ensure it's a string
df["Cacao Percent"] = df["Cacao Percent"].str.replace("%", "").astype(float) / 100
print("Transformation Complete!\n")

Transforming 'Cacao Percent' column...
Transformation Complete!



In [8]:
## 2. Split 'Ingredients' into Multiple Columns
print("Splitting 'Ingredients' column...")
ingredients = df["Ingredients"].str.extract(r'(?P<ingredient_count>\d+)-? ?(?P<ingredients>[A-Z,]*)')
df["ingredient_count"] = ingredients["ingredient_count"].astype(float)

# Ingredient types from website: B, C, L, S, S*, Sa, V
ingredient_cols = ["B", "C", "L", "S", "S*", "Sa", "V"]
ing_df = ingredients["ingredients"].str.get_dummies(sep=",")

# Ensure all columns exist
ing_df = ing_df.reindex(columns=ingredient_cols, fill_value=0)

# Merge back into DataFrame
df = pd.concat([df, ing_df], axis=1)
print("Ingredients Column Split and Added!\n")

Splitting 'Ingredients' column...
Ingredients Column Split and Added!



In [9]:
## 3. Expand 'Most Memorable Characteristics' into Individual Columns
print("Processing 'Most Memorable Characteristics' column...")
characteristic_counts = df["Most Memorable Characteristics"].str.get_dummies(sep=",").sum().sort_values(ascending=False)
selected_characteristics = characteristic_counts[characteristic_counts >= 20].index.tolist()
char_df = df["Most Memorable Characteristics"].str.get_dummies(sep=",")[selected_characteristics]
df = pd.concat([df, char_df], axis=1)
print("Characteristics Column Processed!\n")

Processing 'Most Memorable Characteristics' column...
Characteristics Column Processed!



In [10]:
# Drop unnecessary columns
df.drop(columns=["Ingredients", "Most Memorable Characteristics"], inplace=True)


### 3. Save Cleaned Data

In [11]:
cleaned_file = "data/cleaned_data_2025_flavors_of_cacao.csv"
df.to_csv(cleaned_file, index=False)
print(f"Cleaned dataset saved as {cleaned_file}\n")


Cleaned dataset saved as data/cleaned_data_2025_flavors_of_cacao.csv



### 4.Extract, Transform & Export JSON/CSV

In [12]:
## Select Required Columns
data_reduced = df[["Review Date", "Country of Bean Origin", "Cacao Percent", "Rating"]]

# Save CSV and JSON
reduced_csv = "data/data_reduced_2025_flavors_of_cacao.csv"
reduced_json = "data/data_reduced_2025_flavors_of_cacao.json"
data_reduced.to_csv(reduced_csv, index=False)
data_reduced.to_json(reduced_json, orient="records", indent=4)
print(f"Reduced dataset saved as {reduced_csv} and {reduced_json}\n")

Reduced dataset saved as data/data_reduced_2025_flavors_of_cacao.csv and data/data_reduced_2025_flavors_of_cacao.json



### 5.Filter, Transform & Export Filtered Data

In [13]:
## Apply Filtering Conditions
print("Filtering dataset based on conditions...")
data_filtered = df[
    (df["Rating"] >= 3.25) &
    (df["Cacao Percent"].between(0.65, 0.75)) &
    (df["Review Date"].between(2018, 2021)) &
    (
        df[["fatty", "earthy", "roasty"]].sum(axis=1) > 0
    )
]
print("Data filtered successfully!\n")

Filtering dataset based on conditions...
Data filtered successfully!



In [14]:
# Save Filtered Data
filtered_csv = "data/data_filtered_2025_flavors_of_cacao.csv"
filtered_json = "data/data_filtered_2025_flavors_of_cacao.json"
data_filtered.to_csv(filtered_csv, index=False)
data_filtered.to_json(filtered_json, orient="records", indent=4)
print(f"Filtered dataset saved as {filtered_csv} and {filtered_json}\n")

print("All tasks completed successfully!")

Filtered dataset saved as data/data_filtered_2025_flavors_of_cacao.csv and data/data_filtered_2025_flavors_of_cacao.json

All tasks completed successfully!
